In [ ]:
import numpy as np
model_path = "W:/Models/Sentence Embeddings/universal sentence encoder"


In [ ]:
from tf_emb import TensorflowHubEmbeddings
embedding = TensorflowHubEmbeddings()

In [ ]:
s1 = "i like dogs"
s2 = "i like cannies"
s3 = "i like candies"
s4 = "the weather is ugly outside"
e = np.array(model([s1, s2, s3, s4]))
print(e)
print(np.dot(e[0], e[2]))

In [8]:
import os
from langchain.document_loaders import PyPDFLoader
from tf_emb import TensorflowHubEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Typesense
embedding = TensorflowHubEmbeddings()
loader = PyPDFLoader("PDF/DPO.pdf")
pages = loader.load()
len(pages)

27

In [9]:
pages[0].page_content

'Direct Preference Optimization:\nYour Language Model is Secretly a Reward Model\nRafael Rafailov∗†Archit Sharma∗†Eric Mitchell∗†\nStefano Ermon†‡Christopher D. Manning†Chelsea Finn†\n†Stanford University‡CZ Biohub\n{rafailov,architsh,eric.mitchell}@cs.stanford.edu\nAbstract\nWhile large-scale unsupervised language models (LMs) learn broad world knowl-\nedge and some reasoning skills, achieving precise control of their behavior is\ndifficult due to the completely unsupervised nature of their training. Existing\nmethods for gaining such steerability collect human labels of the relative quality of\nmodel generations and fine-tune the unsupervised LM to align with these prefer-\nences, often with reinforcement learning from human feedback (RLHF). However,\nRLHF is a complex and often unstable procedure, first fitting a reward model that\nreflects the human preferences, and then fine-tuning the large unsupervised LM\nusing reinforcement learning to maximize this estimated reward without dr

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap=30)
docs = splitter.split_documents(pages)
len(docs)

1216

In [11]:
docsearch = Typesense.from_documents(
    docs,
    embedding,
    typesense_client_params={
        "host": "localhost",  # Use xxx.a1.typesense.net for Typesense Cloud
        "port": "8108",  # Use 443 for Typesense Cloud
        "protocol": "http",  # Use https for Typesense Cloud
        "typesense_api_key": "xyz",
        "typesense_collection_name": "lang-chain",
    },
)

# docker run -p 8108:8108 -v %cd%/typesense-data:/data typesense/typesense:26.0 --data-dir /data --api-key="xyz" --enable-cors

In [12]:
query = "What is idea behind direct preference optimization"
found_docs = docsearch.similarity_search(query)
print(found_docs)

[Document(page_content='he sang befor e Emper or Akbar , who was so impr essed by', metadata={'page': 2, 'source': 'Docs\\fepw104.pdf'}), Document(page_content='he sang befor e Emper or Akbar , who was so impr essed by', metadata={'page': 2, 'source': 'Docs\\fepw104.pdf'}), Document(page_content='he sang befor e Emper or Akbar , who was so impr essed by', metadata={'page': 2, 'source': 'Docs\\fepw104.pdf'}), Document(page_content='he sang befor e Emper or Akbar , who was so impr essed by', metadata={'page': 2, 'source': 'Docs\\fepw104.pdf'}), Document(page_content='he sang befor e Emper or Akbar , who was so impr essed by', metadata={'page': 2, 'source': 'Docs\\fepw104.pdf'}), Document(page_content='he sang befor e Emper or Akbar , who was so impr essed by', metadata={'page': 2, 'source': 'Docs\\fepw104.pdf'}), Document(page_content='he sang befor e Emper or Akbar , who was so impr essed by', metadata={'page': 2, 'source': 'Docs\\fepw104.pdf'}), Document(page_content='he sang befor e E

In [ ]:
import typesense

client = typesense.Client({
  'nodes': [{
    'host': 'localhost',
    'port': '8108',
    'protocol': 'http'
  }],
  'api_key': 'xyz',
  'connection_timeout_seconds': 2
})


key = client.keys.create({
  "description": "Admin key.",
  "actions": ["*"],
  "collections": ["*"]
})




In [ ]:
from langchain_community.vectorstores import Chroma
persist_directory = 'docs/chroma/'
# !rm -rf ./docs/chroma

In [ ]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
question = "what is disadvantage of reinforcement learning form human feedback"
docs = vectordb.similarity_search(question,k=3)

In [ ]:
for d in docs:
    print(d.metadata)
    print("\n")

In [ ]:
# to save
vectordb.persist()